In [60]:
import json

with open('../data/annotation_data/longform_model_editing_annotation_pretest_annotations.json') as f:
    pretest = json.load(f)

In [83]:
raters = {

}
rating_to_number = {
    'inconsistent': 0,
    'neutral': 1,
    'consistent': 2
}
agreements = {
    1: 0,
    2: 0,
    3: 0,
    4: 0
}
agreements_by_class = {
    'inconsistent': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    },
    'neutral': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    },
    'consistent': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    }
}
for example in pretest['examples']:
    agreements[
        len(example['classifications'])
    ] += 1
    for rating in example['classifications']:
        for rater in rating['classified_by']:
            anon_id = rater['annotator_id']
            if anon_id not in raters:
                raters[anon_id] = []
            raters[anon_id].append(rating_to_number[rating['classname']])
            agreements_by_class[rating['classname']][len(example['classifications'])] += 1

In [85]:
agreements

114

In [81]:
import krippendorff as kd
kd.alpha([
    list(raters.values())[0],
    list(raters.values())[1],
    list(raters.values())[2],
    list(raters.values())[3]
])


1.0

In [73]:
# cohens kappa
from sklearn.metrics import cohen_kappa_score

for rater_1 in range(4):
    for rater_2 in range(4):
        if rater_1 == rater_2:
            continue
        print(cohen_kappa_score(
            list(raters.values())[rater_1],
            list(raters.values())[rater_2]
        ))

0.45622119815668205
0.47577398306261276
0.45386172982354644
0.45622119815668205
0.507725587144623
0.5574037834311807
0.47577398306261276
0.507725587144623
0.6122448979591837
0.45386172982354656
0.5574037834311807
0.6122448979591837


In [30]:
import json
import pandas as pd

gpt_35_annotations_human_path = '../results/annotations_llama2_chat_human_edit_pretest_annotation_gpt-3.5-turbo-0613.json'
gpt_35_annotations_human = json.load(open(gpt_35_annotations_human_path))

gpt_35_annotations_no_edit_path = '../results/annotations_llama2_chat_no_edit_pretest_annotation_gpt-3.5-turbo-0613.json'
gpt_35_annotations_no_edit = json.load(open(gpt_35_annotations_no_edit_path))

gpt_35_annotations_edit_path = '../results/annotations_llama2_chat_rome_edit_pretest_annotation_gpt-3.5-turbo-0613.json'
gpt_35_annotations_edit = json.load(open(gpt_35_annotations_edit_path))

gpt_35_df = pd.DataFrame(
    gpt_35_annotations_human + gpt_35_annotations_no_edit + gpt_35_annotations_edit
)

gpt_4_annotations_human_path = '../results/annotations_llama2_chat_human_edit_pretest_annotation_gpt-4.json'
gpt_4_annotations_human = json.load(open(gpt_4_annotations_human_path))

gpt_4_annotations_no_edit_path = '../results/annotations_llama2_chat_no_edit_pretest_annotation_gpt-4.json'
gpt_4_annotations_no_edit = json.load(open(gpt_4_annotations_no_edit_path))

gpt_4_annotations_edit_path = '../results/annotations_llama2_chat_rome_edit_pretest_annotation_gpt-4.json'
gpt_4_annotations_edit = json.load(open(gpt_4_annotations_edit_path))

gpt_4_df = pd.DataFrame(
    gpt_4_annotations_human + gpt_4_annotations_no_edit + gpt_4_annotations_edit
)





In [50]:
import json

with open('../data/annotation_data/longform_eval_first_3_samples_paragraph_annotations.json') as f:
    pretest = json.load(f)

intervention_map = {
    'human': 'llama2_chat_human_edit_pretest_annotation',
    'no_edit': 'llama2_chat_no_edit_pretest_annotation',
    'rome': 'llama2_chat_rome_edit_pretest_annotation'
}

raters = {
    1: [],
    2: []
}
rating_to_number = {
    'supports': 0,
    'neutral': 1,
    'contradicts': 2
}
agreements = {
    1: 0,
    2: 0,
    3: 0,
    4: 0
}
agreements_by_class = {
    'contradicts': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    },
    'neutral': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    },
    'supports': {
        1: 0,
        2: 0,
        3: 0,
        4: 0
    }
}
for example in pretest['examples']:
    agreements[
        len(example['classifications'])
    ] += 1
    for rating in example['classifications']:
        for rater in rating['classified_by']:
            anon_id = rater['annotator_id']
            if anon_id not in raters:
                raters[anon_id] = []
            
            agreements_by_class[rating['classname']][len(example['classifications'])] += 1
            try:
                raters[2].append(
                    gpt_35_df.loc[
                        (gpt_35_df['content'] == example['content'])
                    ]['classification'].values[0]
                )
                raters[anon_id].append(rating['classname'])
            except:
                continue

In [51]:

import numpy as np

np.sum([
    raters[1][i] == raters[2][i] for i in range(len(raters[1]))
]) / len(raters[1])

0.46774193548387094

In [52]:
# cohens kappa

from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(
    raters[1],
    raters[2]
)

0.2473021582733813